In [4]:
import numpy as np
from scipy import misc
from scipy import linalg
from matplotlib import pylab as plt
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation as clf
from sklearn import metrics as met
import matplotlib.cm as cm

In [5]:
#train data
train_labels, X = [], []
for line in open("faces/train.txt"):
    im = misc.imread(line.strip().split()[0])
    X.append(im.reshape(2500,))
    train_labels.append(line.strip().split()[1])
X, train_labels = np.array(X, dtype=float), np.array(train_labels, dtype=int)
print X.shape, train_labels.shape
plt.imshow(X[10, :].reshape(50,50), cmap = cm.Greys_r)
plt.show()

(540, 2500) (540,)


In [6]:
#test data
test_labels, test_data=[], []

for line2 in open("faces/test.txt"):
    im = misc.imread(line2.strip().split()[0])
    test_data.append(im.reshape(2500,))
    test_labels.append(line2.strip().split()[1])
test_data, test_labels = np.array(test_data, dtype=float), np.array(test_labels, dtype=int)

print test_data.shape, test_labels.shape
plt.imshow(test_data[10, :].reshape(50,50), cmap = cm.Greys_r)
plt.show()

(100, 2500) (100,)


In [7]:
#average face
avgFace=np.float()
avgFace=np.zeros(2500);
i=0
j=0
while j<2500:
    while(i<540) :
        avgFace[j]+= X[i][j]
        i+=1
    i=0
    j+=1
j=0
while(j<2500):
    avgFace[j]=avgFace[j]/ 540
    j+=1
plt.imshow(avgFace.reshape(50,50), cmap = cm.Greys_r)
plt.show()

In [8]:
#mean substraction
meanSub= np.zeros(2500)
meanSub= np.array(X,dtype=float)
k=0
l=0
while(k<540):
    arr = np.float()
    arr=np.zeros(2500)
    while(l<2500):
        meanSub[k][l]= meanSub[k][l]- avgFace[l]
        l+=1
    l=0
    k+=1
plt.imshow(meanSub[10,:].reshape(50,50), cmap = cm.Greys_r)
plt.show()

meanSubTest=np.float()
meanSubTest= np.zeros(2500)
meanSubTest= np.array(test_data,dtype=int)
k=0
l=0
while k<100:
    while(l<2500):
        meanSubTest[k][l]= test_data[k][l]- avgFace[l]
        l+=1
    l=0
    k+=1
plt.imshow(meanSubTest[10, :].reshape(50,50), cmap = cm.Greys_r)
plt.show()

In [9]:
#SVD
u,sig,v = linalg.svd(meanSub, full_matrices= True)
i=0
for i in range(0,10):
    plt.imshow(v[i,:].reshape(50,50), cmap = cm.Greys_r)
    plt.show()
    plt.savefig("pic"+str(i)+".jpg")

In [10]:
#low rank approximation
error = np.float()
error = np.zeros(200)
sig=sorted(sig,reverse=True)
for k in range(0,200):
    lowRank=np.dot(np.dot(u[:,:k],np.diag(sig[:k])),v[:k,:])
    error[k]=np.linalg.norm(X-lowRank)

plt.plot(error)
plt.show()

In [11]:
#eigenface feature
def trainEigenFaceFeature(r):
    Ft = np.dot(meanSub, np.transpose(v[:r,:]))
    return Ft

def testEigenFaceFeature(r):
    m = np.dot(meanSubTest, np.transpose(v[:r,:]))
    return m

F=trainEigenFaceFeature(10)
F_test=testEigenFaceFeature(10)

In [12]:
#face recognition
model= LogisticRegression()
model = model.fit(F,train_labels)
print model.score(F_test,test_labels)

prediction=model.predict(F_test)
with open("prediction.txt","w") as textfile:
    for ele in prediction:
        textfile.write(str(ele)+"\n")

acc = np.float()
acc= np.zeros(200)
for i in range(1,200):
    F=trainEigenFaceFeature(i)
    model = model.fit(F,train_labels)
    F_test=testEigenFaceFeature(i)
    predicted = model.predict(F_test)
    labels = []
    for obj in predicted:
        labels.append(str(obj))
    labels = np.array(labels, dtype=int)
    acc[i] = met.accuracy_score(labels,test_labels)
plt.plot(acc)
plt.show()

0.79
